# Fastspeech2 해설 및 실습

In [1]:
import tensorflow as tf
print(tf.__version__)

2.6.0


In [2]:
# -*- coding: utf-8 -*-
# Copyright 2020 The FastSpeech2 Authors and Minh Nguyen (@dathudeptrai)
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""Tensorflow Model modules for FastSpeech2."""

'Tensorflow Model modules for FastSpeech2.'

In [3]:
!pip install TensorFlowTTS

     |████████████████████████████████| 128 kB 5.4 MB/s 
     |████████████████████████████████| 3.1 MB 28.9 MB/s 
     |████████████████████████████████| 458.3 MB 10 kB/s 
     |████████████████████████████████| 212 kB 41.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.7 MB 38.7 MB/s 
     |████████████████████████████████| 1.3 MB 39.3 MB/s 
     |████████████████████████████████| 3.5 MB 32.5 MB/s 
     |████████████████████████████████| 1.1 MB 45.2 MB/s 
     |████████████████████████████████| 235 kB 43.0 MB/s 
     |████████████████████████████████| 20.2 MB 1.6 MB/s 
     |████████████████████████████████| 180 kB 41.1 MB/s 
  Created wheel for pyworld: filename=pyworld-0.3.0-cp37-cp37m-linux_x86_64.whl size=609651 sha256=15dbdba6a237a0c7e6eeb7a6d408d565bf7c7789e6edee943c7fc4f14ec63c12
  Stored in directory: /root/.cache/pip/wheels/e7/7c/11/c775fffa0e1e7b05a

In [4]:
# Mount Google Drive to VM instance.
from google.colab import drive
drive.mount('/content/drive')

!git clone https://github.com/TensorSpeech/TensorFlowTTS.git

Mounted at /content/drive
Cloning into 'TensorFlowTTS'...
remote: Enumerating objects: 10609, done.
remote: Counting objects: 100% (319/319), done.
remote: Compressing objects: 100% (162/162), done.
remote: Total 10609 (delta 183), reused 233 (delta 152), pack-reused 10290
Receiving objects: 100% (10609/10609), 133.30 MiB | 27.18 MiB/s, done.
Resolving deltas: 100% (5123/5123), done.


In [5]:
!pip install h5py==2.10
!pip install git+https://github.com/repodiac/german_transliterate
"""
HDF5(계층적데이터형식, 빅데이터 처리를 위한 파일 형식)을 python에서 사용하기 위한 모듈
독일어 자역 모듈
약어, 숫자, 타임 스탬프 등을 포함한 독일어 텍스트를 정리하고 음역 (예 : 정규화)하는 Python 모듈
"""

     |████████████████████████████████| 2.9 MB 5.5 MB/s 
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.6.0 requires h5py~=3.1.0, but you have h5py 2.10.0 which is incompatible.
tensorflow-gpu 2.6.0 requires h5py~=3.1.0, but you have h5py 2.10.0 which is incompatible.


  Cloning https://github.com/repodiac/german_transliterate to /tmp/pip-req-build-7dzisb62
  Running command git clone -q https://github.com/repodiac/german_transliterate /tmp/pip-req-build-7dzisb62
     |████████████████████████████████| 101 kB 4.4 MB/s 
  Created wheel for german-transliterate: filename=german_transliterate-0.1.3-py3-none-any.whl size=20830 sha256=7aa948466b4350274de085a1e052391677c52a8defea0f361b6992955dff0516
  Stored in directory: /tmp/pip-ephem-wheel-cache-8amlmusx/wheels/67/99/62/1f618221d7d4275595cf736d3e2d587b64bf427571c9b9a84d
Successfully built german-transliterate


'\nHDF5(계층적데이터형식, 빅데이터 처리를 위한 파일 형식)을 python에서 사용하기 위한 모듈\n독일어 자역 모듈\n약어, 숫자, 타임 스탬프 등을 포함한 독일어 텍스트를 정리하고 음역 (예 : 정규화)하는 Python 모듈\n'

In [7]:
import tensorflow as tf
from tensorflow_tts.models.fastspeech import TFFastSpeech, get_initializer

## TFFastSpeechVariantPredictor 클래스

In [8]:
# fastspeech 예측 모델 (keras.layers.Layer에서 상속받은 객체)
class TFFastSpeechVariantPredictor(tf.keras.layers.Layer):
    """FastSpeech duration predictor module."""

    def __init__(self, config, **kwargs):
        """Init variables."""
        super().__init__(**kwargs)
        # 상속받은 class의 생성자를 호출하기 위해서 super를 씀
        
        self.conv_layers = []
        
        # 합성곱 레이어 생성 
        for i in range(config.variant_prediction_num_conv_layers):
            # config.variant_prediction_num_conv_layers의 숫자만큼
            self.conv_layers.append(
                tf.keras.layers.Conv1D(
                    config.variant_predictor_filter, # Conv 필터
                    config.variant_predictor_kernel_size, # kernel 사이즈 값
                    padding="same", # 패딩은 입출력이 동일하게 만듬
                    name="conv_._{}".format(i),
                )
            )
            self.conv_layers.append(tf.keras.layers.Activation(tf.nn.relu))
            # 활성함수는 relu 사용

            self.conv_layers.append(
                tf.keras.layers.LayerNormalization(
                    # 정규화 설정
                    epsilon=config.layer_norm_eps, name="LayerNorm_._{}".format(i)
                    # epsilon =  0으로 나누는 것을 피하기 위해 분산에 작은 부동 소수점이 추가
                )
            )
            self.conv_layers.append(
                tf.keras.layers.Dropout(config.variant_predictor_dropout_rate)
                # Dropout 설정
            )
        self.conv_layers_sequence = tf.keras.Sequential(self.conv_layers)
        # conv_sequence에 Sequential객체 담음

        self.output_layer = tf.keras.layers.Dense(1)
        # 마지막 output_layer에는 하나의 Dense레이어를 삽입

        if config.n_speakers > 1:
            # 만약 n_speakers가 있다면 임베딩 벡터를 만들기위한 임베딩 레이어 삽입
            
            # 디코더에 들어갈 임베딩레이어
            self.decoder_speaker_embeddings = tf.keras.layers.Embedding(
                config.n_speakers,
                config.encoder_self_attention_params.hidden_size,
                # 히든 스테이트 사이즈 지정
                embeddings_initializer=get_initializer(config.initializer_range),
                # 임베딩 초기화 (설정값대로?)
                name="speaker_embeddings",
            )
            # speaker_fc? Dense 레이어에 self_attention의 히든 스테이트 사이즈 지정
            self.speaker_fc = tf.keras.layers.Dense(
                units=config.encoder_self_attention_params.hidden_size,
                name="speaker_fc",
            )
        # 환경 설정
        self.config = config

## Call 함수

In [9]:
    # 로직 불러오기 (call 메서드)
    def call(self, inputs, training=False):
        # 새로 학습하지 않고 사용 training=False

        """Call logic."""
        encoder_hidden_states, speaker_ids, attention_mask = inputs
        # encoder_hidden_states, speaker_ids, attention_mask 지정
        # speaker_ids 는 화자 아이디

        attention_mask = tf.cast(
            tf.expand_dims(attention_mask, 2), encoder_hidden_states.dtype
            # attantion_mask에 2차원 추가하고 encoder_hidden_states와 dtype 맞춤
        )

        if self.config.n_speakers > 1:
            speaker_embeddings = self.decoder_speaker_embeddings(speaker_ids)
            # 임베딩 벡터화?
            speaker_features = tf.math.softplus(self.speaker_fc(speaker_embeddings))
            # softplus계산을 통해 특성을 뽑아냄

            # extended speaker embeddings
            # speaker 임베딩을 확장
            extended_speaker_features = speaker_features[:, tf.newaxis, :]
            # speaker_features 1차원 추가

            encoder_hidden_states += extended_speaker_features
            # speaker_features 만큼 hidden_states 추가

        # mask encoder hidden states
        masked_encoder_hidden_states = encoder_hidden_states * attention_mask
        # encoder_hidden_states * attention_mask을 엘레먼트 와이즈하여 mask를 씌워줌

        # pass though first layer
        # 첫번째 층 통과
        outputs = self.conv_layers_sequence(masked_encoder_hidden_states)
        outputs = self.output_layer(outputs)
        masked_outputs = outputs * attention_mask
        # outputs 에 mask를 씌워줌

        print('masked_outputs : ', masked_outputs.shape)
        outputs = tf.squeeze(masked_outputs, -1)
        print('outputs : ', outputs.shape)
        # -1 치수 값 제거
        return outputs

## TFFastSpeech2 클래스

In [10]:
# TFFastSpeech를 상속받은 FastSpeech2 클래스
class TFFastSpeech2(TFFastSpeech):
    """TF Fastspeech module."""

    def __init__(self, config, **kwargs):
        """Init layers for fastspeech."""
        super().__init__(config, **kwargs)
        self.f0_predictor = TFFastSpeechVariantPredictor(
            config, dtype=tf.float32, name="f0_predictor"
        ) # f0_predictor = TFFastSpeechVariantPredictor 상속

        self.energy_predictor = TFFastSpeechVariantPredictor(
            config, dtype=tf.float32, name="energy_predictor",
        ) # energy_predictor = TFFastSpeechVariantPredictor 상속

        self.duration_predictor = TFFastSpeechVariantPredictor(
            config, dtype=tf.float32, name="duration_predictor"
        ) # duration_predictor = TFFastSpeechVariantPredictor 상속

        # define f0_embeddings and energy_embeddings
        # f0_embeddings 과 energy_embeddings를 정의
        self.f0_embeddings = tf.keras.layers.Conv1D(
            filters=config.encoder_self_attention_params.hidden_size,
            kernel_size=9,
            padding="same",
            name="f0_embeddings",
        ) # f0_embeddings에 Conv레이어 삽입

        self.f0_dropout = tf.keras.layers.Dropout(0.5)
        # Dropout은 50%

        self.energy_embeddings = tf.keras.layers.Conv1D(
            filters=config.encoder_self_attention_params.hidden_size,
            kernel_size=9,
            padding="same",
            name="energy_embeddings",
        ) # energy_embeddings Conv 레이어 삽입

        self.energy_dropout = tf.keras.layers.Dropout(0.5)
        # Dropout은 50%

## build 함수

In [11]:
    # 빌드 메서드
    def _build(self):
        """Dummy input for building model."""
        # fake inputs
        input_ids = tf.convert_to_tensor([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]], tf.int32)
        # value값을 tensor로 변환, input_ids값 지정
        
        speaker_ids = tf.convert_to_tensor([0], tf.int32)
        # speaker_ids값 지정

        duration_gts = tf.convert_to_tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], tf.int32)
        # duration_gts 지정

        f0_gts = tf.convert_to_tensor(
            [[10, 10, 10, 10, 10, 10, 10, 10, 10, 10]], tf.float32
        ) # f0_gts 지정

        energy_gts = tf.convert_to_tensor(
            [[10, 10, 10, 10, 10, 10, 10, 10, 10, 10]], tf.float32
        ) # energy_gts 지정

        self(
            input_ids=input_ids,
            speaker_ids=speaker_ids,
            duration_gts=duration_gts,
            f0_gts=f0_gts,
            energy_gts=energy_gts,
        )

## call 함수

In [20]:
    def call( # 들어가는 변수가 무엇인지 모르겟음
        self,
        input_ids,
        speaker_ids, # 화자의 수 
        duration_gts,
        f0_gts,
        energy_gts,
        training=False,
        **kwargs,
    ):
        """Call logic."""
        attention_mask = tf.math.not_equal(input_ids, 0)
        # input_ids 와 0을 비교하여 True, False값으로 attention_mask 만듬

        embedding_output = self.embeddings([input_ids, speaker_ids], training=training)
        # 임베딩 output

        encoder_output = self.encoder(
            [embedding_output, attention_mask], training=training
        ) # 인코더 output

        last_encoder_hidden_states = encoder_output[0]
        # encoder_output의 열개수로 last_encoder_hidden_states를 지정

        # energy predictor, here use last_encoder_hidden_states, u can use more hidden_states layers
        # energy predictor, 여기서는 last_contain_hidden_states 사용,  더 많은 hidden_states 레이어를 사용할 수 있습니다.

        # rather than just use last_hidden_states of encoder for energy_predictor.
        # energy_didden_states를 위해 인코더의 last_hidden_states만 사용하는 것이 아니라
        duration_outputs = self.duration_predictor(
            [last_encoder_hidden_states, speaker_ids, attention_mask]
        )  # [batch_size, length]
        # duration_predictor 통해 duration_outputs 지정

        f0_outputs = self.f0_predictor(
            [last_encoder_hidden_states, speaker_ids, attention_mask], training=training
        ) # f0_predictor로 f0_outputs 지정

        energy_outputs = self.energy_predictor(
            [last_encoder_hidden_states, speaker_ids, attention_mask], training=training
        ) # energy_predictor로 energy_outputs 지정

        f0_embedding = self.f0_embeddings(
            tf.expand_dims(f0_gts, 2)
            # f0_gts, 2차원 추가
        )  # [barch_size, mel_length, feature]

        energy_embedding = self.energy_embeddings(
            tf.expand_dims(energy_gts, 2)
            # energy_gts, 2차원 추가
        )  # [barch_size, mel_length, feature]

        # apply dropout both training/inference
        # training/inference 둘다 dropout 적용
        f0_embedding = self.f0_dropout(f0_embedding, training=True)
        energy_embedding = self.energy_dropout(energy_embedding, training=True)

        # sum features
        # 특성값의 합
        last_encoder_hidden_states += f0_embedding + energy_embedding
        # f0_embedding + energy_embedding 값을 last_encoder_hidden_states에 추가

        # 범위 조절 output
        length_regulator_outputs, encoder_masks = self.length_regulator(
            [last_encoder_hidden_states, duration_gts], training=training
        )

        # create decoder positional embedding
        # 디코더 포지션임베딩 생성
        decoder_pos = tf.range(
            1, tf.shape(length_regulator_outputs)[1] + 1, dtype=tf.int32
        )
        masked_decoder_pos = tf.expand_dims(decoder_pos, 0) * encoder_masks
        # decoder_pos에 차원을 늘리고 mask를 씌움

        decoder_output = self.decoder(
            [length_regulator_outputs, speaker_ids, encoder_masks, masked_decoder_pos],
            training=training,
        ) # 디코더의 output을 지정

        last_decoder_hidden_states = decoder_output[0]
        # last_decoder_hidden_states는 디코더의 output의 열개수 만큼으로 지정

        # here u can use sum or concat more than 1 hidden states layers from decoder.
        # 여기서 당신은 디코더에서 둘 이상의 숨겨진 상태 레이어를 사용할 수 있다.
        mels_before = self.mel_dense(last_decoder_hidden_states)
        mels_after = (
            self.postnet([mels_before, encoder_masks], training=training) + mels_before
        )

        # ouptputs 지정
        outputs = (
            mels_before,
            mels_after,
            duration_outputs,
            f0_outputs,
            energy_outputs,
        )
        return outputs

## inference 함수 (추론)

In [21]:
    # _inference 메서드
    def _inference(
        self,
        input_ids, # input 값
        speaker_ids, # 화자의 수
        speed_ratios, # 속도율
        f0_ratios,
        energy_ratios, # 에너지율
        **kwargs,
    ):
        """Call logic."""
        attention_mask = tf.math.not_equal(input_ids, 0)
        # input_ids, 0을 비교하여 True, False값으로 attention_mask 생성

        embedding_output = self.embeddings([input_ids, speaker_ids], training=False)
        # input값과 화자의수로 임베딩

        encoder_output = self.encoder(
            [embedding_output, attention_mask], training=False
        ) # 임베딩된 값과 mask한 값을 인코딩함

        last_encoder_hidden_states = encoder_output[0]
        # last_encoder_hidden_states는 인코더의 output의 열개수로 지정

        # expand ratios
        # 비율 확장
        speed_ratios = tf.expand_dims(speed_ratios, 1)  # [B, 1]
        f0_ratios = tf.expand_dims(f0_ratios, 1)  # [B, 1]
        energy_ratios = tf.expand_dims(energy_ratios, 1)  # [B, 1]

        # energy predictor, here use last_encoder_hidden_states, u can use more hidden_states layers
        # energy predictor, 여기서는 last_contain_hidden_states 사용, 더 많은 hidden_states 레이어를 사용할 수 있습니다.

        # rather than just use last_hidden_states of encoder for energy_predictor.
        # energy_didden_states를 위해 인코더의 last_hidden_states만 사용하는 것이 아니라

        duration_outputs = self.duration_predictor(
            # 지속 시간 예측

            [last_encoder_hidden_states, speaker_ids, attention_mask]
        )  # [batch_size, length]

        duration_outputs = tf.nn.relu(tf.math.exp(duration_outputs) - 1.0)
        # 지수 값을 계산 - 1.0 을 relu로 계산

        duration_outputs = tf.cast(
            tf.math.round(duration_outputs * speed_ratios), tf.int32
        ) # (duration_outputs * speed_ratios) 반올림하여 int형변환

        f0_outputs = self.f0_predictor(
            [last_encoder_hidden_states, speaker_ids, attention_mask], training=False
        ) # f0_outputs 지정
        
        f0_outputs *= f0_ratios
        # f0_outputs에 f0_ratios를 곱해줌
        
        energy_outputs = self.energy_predictor(
            [last_encoder_hidden_states, speaker_ids, attention_mask], training=False
        ) # energy_outputs 지정

        energy_outputs *= energy_ratios
        # energy_outputs에 energy_ratios를 곱해줌

        f0_embedding = self.f0_dropout(
            self.f0_embeddings(tf.expand_dims(f0_outputs, 2)), training=True
        ) # f0_embedding 지정 (dropout)

        energy_embedding = self.energy_dropout(
            self.energy_embeddings(tf.expand_dims(energy_outputs, 2)), training=True
        ) # energy_embedding 지정 (dropout)

        # sum features
        last_encoder_hidden_states += f0_embedding + energy_embedding
        # f0_embedding + energy_embedding 값을 last_encoder_hidden_states 추가

        length_regulator_outputs, encoder_masks = self.length_regulator(
            [last_encoder_hidden_states, duration_outputs], training=False
        ) # 길이 조절

        # create decoder positional embedding
        # 디코더 포지션임베딩 생성
        decoder_pos = tf.range(
            1, tf.shape(length_regulator_outputs)[1] + 1, dtype=tf.int32
        ) # decoder_pos 지정

        masked_decoder_pos = tf.expand_dims(decoder_pos, 0) * encoder_masks
        # decoder_pos 에 mask 씌움

        decoder_output = self.decoder(
            [length_regulator_outputs, speaker_ids, encoder_masks, masked_decoder_pos],
            training=False,
        ) # 디코더 진행

        last_decoder_hidden_states = decoder_output[0]
        # last_decoder_hidden_states는 디코더output의 열개수

        # here u can use sum or concat more than 1 hidden states layers from decoder.
        # 여기서 당신은 디코더에서 둘 이상의 숨겨진 상태 레이어를 사용할 수 있다.
        mel_before = self.mel_dense(last_decoder_hidden_states)
        mel_after = (
            self.postnet([mel_before, encoder_masks], training=False) + mel_before
        ) # postnet + mel_dense로 mel_after 생성

        outputs = (mel_before, mel_after, duration_outputs, f0_outputs, energy_outputs)
        return outputs

## setup_inference_fn 함수

In [19]:
    # setup_inference_fn 메서드
    def setup_inference_fn(self):
        self.inference = tf.function( # tf.function 함수를 활용하여 호출 가능한 TensorFlow 그래프로 컴파일합니다.
            self._inference,
            experimental_relax_shapes=True,
            # True면, tf.function은 더 적게 즉, 입력 모양에서 덜 다루어지는 그래프를 생성

            input_signature=[ 
                # 입력 시그니쳐(signature)에 대해 초기화 (텐서의 모양으로 전달)
                tf.TensorSpec(shape=[None, None], dtype=tf.int32, name="input_ids"),
                tf.TensorSpec(shape=[None,], dtype=tf.int32, name="speaker_ids"),
                tf.TensorSpec(shape=[None,], dtype=tf.float32, name="speed_ratios"),
                tf.TensorSpec(shape=[None,], dtype=tf.float32, name="f0_ratios"),
                tf.TensorSpec(shape=[None,], dtype=tf.float32, name="energy_ratios"),
            ],
        )

        self.inference_tflite = tf.function(
            self._inference,
            experimental_relax_shapes=True,
            # True면, tf.function은 더 적게 즉, 입력 모양에서 덜 다루어지는 그래프를 생성
            input_signature=[
                # 입력 시그니쳐(signature)에 대해 초기화 (텐서의 모양으로 전달)
                tf.TensorSpec(shape=[1, None], dtype=tf.int32, name="input_ids"),
                tf.TensorSpec(shape=[1,], dtype=tf.int32, name="speaker_ids"),
                tf.TensorSpec(shape=[1,], dtype=tf.float32, name="speed_ratios"),
                tf.TensorSpec(shape=[1,], dtype=tf.float32, name="f0_ratios"),
                tf.TensorSpec(shape=[1,], dtype=tf.float32, name="energy_ratios"),
            ],
        )